## Produce daily Origin-Destination trip matrix for NYC Resident borough to all U.S. counties using csv generated by EDM 

### Separates count for trips where origin == destination (i.e. "Home visit"), and all other visits

In [5]:
import pandas as pd
import numpy as np
import os
import time

In [6]:
from geo import stco,sub

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
df = pd.read_csv('../data/csvs/daily_nycto_us_county_outflow 201009.csv')

In [9]:
df['not_home_visits'] = df['visits'] - df['at_home_visits']

In [10]:
dff = pd.pivot_table(df,values=['not_home_visits'],index=['date_start'],columns=['destination_fips'],aggfunc=np.sum,fill_value=0)
dff.columns = dff.columns.droplevel()
dff.head()

destination_fips  1001   1003   1005   1007   1009   1011   1013   1015   \
date_start                                                                 
2019-01-01            1      5      0      0      0      0      1      3   
2019-01-02            2      5      0      0      0      0      0      7   
2019-01-03            2      2      0      0      0      0      0      2   
2019-01-04            2      3      0      0      1      0      0      0   
2019-01-05            2      2      1      0      1      0      1      1   

destination_fips  1017   1019   1021   1023   1025   1027   1029   1031   \
date_start                                                                 
2019-01-01            0      0      0      0      0      1      0      1   
2019-01-02            0      0      0      0      0      1      0      0   
2019-01-03            1      0      0      0      0      1      4      1   
2019-01-04            1      0      1      0      0      0      0      1   
2019-01-05            0      1      0      0      0      0      0      1   

destination_fips  1033   1035   1037   1039   1041   1043   1045   1047   \
date_start                                                                 
2019-01-01            2      1      0      0      0      5      0      0   
2019-01-02            6      0      0      0      0      7      0      0   
2019-01-03            3      1      0      0      1      7      0      0   
2019-01-04            5      0      0      0      0      1      1      1   
2019-01-05            2      1      0      0      0      1      1      1   

destination_fips  1049   1051   1053   1055   1057   1059   1061   1063   \
date_start                                                                 
2019-01-01            0      2      1      1      0      2      0      0   
2019-01-02            0      2      1      4      0      0      0      0   
2019-01-03            1      1      0      4      0      3      0      0   
2019-01-04            0      2      0      4      1      1      1      0   
2019-01-05            1      4      0      3      0      0      0      2   

destination_fips  1065   1067   1069   1071   1073   1075   1077   1079   \
date_start                                                                 
2019-01-01            0      0      1      0     33      0      0      0   
2019-01-02            0      0      2      0     30      0      0      0   
2019-01-03            0      0      6      0     24      1      0      0   
2019-01-04            0      0     12      2     20      0      0      0   
2019-01-05            0      0      3      1     21      0      0      0   

destination_fips  1081   1083   1085   1087   1089   1091   1093   1095   \
date_start                                                                 
2019-01-01            8      1      0      3     18      2      0      0   
2019-01-02            7      1      0      0     19      1      0      1   
2019-01-03            5      1      0      0     28      3      0      5   
2019-01-04           13      1      0      1     27      2      0      5   
2019-01-05            8      0      0      2     25      3      0      4   

destination_fips  1097   1099   1101   1103   1105   1107   1109   1111   \
date_start                                                                 
2019-01-01           20      0     16      3      0      1      1      0   
2019-01-02           18      0     11      5      0      0      1      0   
2019-01-03           31      0     11      3      0      0      0      0   
2019-01-04           17      0     12      6      0      0      0      0   
2019-01-05           22      0     11      3      0      0      1      0   

destination_fips  1113   1115   1117   1119   1121   1123   1125   1127   \
date_start                                                                 
2019-01-01            1      2      4      1      0      0      4      0   
2019-01-02            1      1      2    

In [11]:
df_home = df[df['at_home_visits'] > 0]

In [12]:
dff_home = pd.pivot_table(df_home,values=['at_home_visits'],index=['date_start'],columns=['destination_fips'],aggfunc=np.sum,fill_value=0,margins=True)
dff_home.columns = dff_home.columns.droplevel()
dff_home = dff_home[['All']].rename(columns={'All':'Home'})
dff_home.head()

destination_fips,Home
date_start,
2019-01-01,430361
2019-01-02,435531
2019-01-03,430425
2019-01-04,433206
2019-01-05,441106


In [20]:
df_device = df[['date_start','origin_fips','device_count','completely_home_device_count']]
df_device = df_device.drop_duplicates(ignore_index=True)

,date_start,origin_fips,device_count,completely_home_device_count
0,2020-04-08,36061,48861,23280
1,2020-04-11,36061,48066,22325
2,2020-04-13,36085,29348,18606
3,2020-03-28,36081,93842,54974
4,2020-05-11,36085,30392,16741


In [23]:
# for NYC total only
dff_device = df_device.groupby('date_start').agg({'device_count':np.sum,'completely_home_device_count':np.sum})

In [14]:
#for county-level only
#dff_device = df_device.set_index('date_start').sort_index(ascending=True)

,device_count,completely_home_device_count
date_start,,
2019-01-01,36884,16432
2019-01-01,109524,55340
2019-01-01,165090,77727
2019-01-01,148363,70510
2019-01-01,86687,36253


In [25]:
df_master = pd.concat([dff_device,dff_home,dff],axis=1)

In [26]:
df_master = df_master.reset_index()

In [27]:
df_master.head()

index  device_count  completely_home_device_count    Home  1001  1003  \
0  2019-01-01      546548.0                      256262.0  430361   1.0   5.0   
1  2019-01-02      548629.0                      220643.0  435531   2.0   5.0   
2  2019-01-03      541106.0                      206784.0  430425   2.0   2.0   
3  2019-01-04      546201.0                      204272.0  433206   2.0   3.0   
4  2019-01-05      548061.0                      255867.0  441106   2.0   2.0   

   1005  1007  1009  1011  1013  1015  1017  1019  1021  1023  1025  1027  \
0   0.0   0.0   0.0   0.0   1.0   3.0   0.0   0.0   0.0   0.0   0.0   1.0   
1   0.0   0.0   0.0   0.0   0.0   7.0   0.0   0.0   0.0   0.0   0.0   1.0   
2   0.0   0.0   0.0   0.0   0.0   2.0   1.0   0.0   0.0   0.0   0.0   1.0   
3   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   1.0   0.0   0.0   0.0   
4   1.0   0.0   1.0   0.0   1.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0   

   1029  1031  1033  1035  1037  1039  1041  1043  1045  1047  1049  1051  \
0   0.0   1.0   2.0   1.0   0.0   0.0   0.0   5.0   0.0   0.0   0.0   2.0   
1   0.0   0.0   6.0   0.0   0.0   0.0   0.0   7.0   0.0   0.0   0.0   2.0   
2   4.0   1.0   3.0   1.0   0.0   0.0   1.0   7.0   0.0   0.0   1.0   1.0   
3   0.0   1.0   5.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0   0.0   2.0   
4   0.0   1.0   2.0   1.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   4.0   

   1053  1055  1057  1059  1061  1063  1065  1067  1069  1071  1073  1075  \
0   1.0   1.0   0.0   2.0   0.0   0.0   0.0   0.0   1.0   0.0  33.0   0.0   
1   1.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0  30.0   0.0   
2   0.0   4.0   0.0   3.0   0.0   0.0   0.0   0.0   6.0   0.0  24.0   1.0   
3   0.0   4.0   1.0   1.0   1.0   0.0   0.0   0.0  12.0   2.0  20.0   0.0   
4   0.0   3.0   0.0   0.0   0.0   2.0   0.0   0.0   3.0   1.0  21.0   0.0   

   1077  1079  1081  1083  1085  1087  1089  1091  1093  1095  1097  1099  \
0   0.0   0.0   8.0   1.0   0.0   3.0  18.0   2.0   0.0   0.0  20.0   0.0   
1   0.0   0.0   7.0   1.0   0.0   0.0  19.0   1.0   0.0   1.0  18.0   0.0   
2   0.0   0.0   5.0   1.0   0.0   0.0  28.0   3.0   0.0   5.0  31.0   0.0   
3   0.0   0.0  13.0   1.0   0.0   1.0  27.0   2.0   0.0   5.0  17.0   0.0   
4   0.0   0.0   8.0   0.0   0.0   2.0  25.0   3.0   0.0   4.0  22.0   0.0   

   1101  1103  1105  1107  1109  1111  1113  1115  1117  1119  1121  1123  \
0  16.0   3.0   0.0   1.0   1.0   0.0   1.0   2.0   4.0   1.0   0.0   0.0   
1  11.0   5.0   0.0   0.0   1.0   0.0   1.0   1.0   2.0   1.0   0.0   0.0   
2  11.0   3.0   0.0   0.0   0.0   0.0   1.0   1.0   4.0   1.0   2.0   1.0   
3  12.0   6.0   0.0   0.0   0.0   0.0   0.0   2.0   6.0   1.0   0.0   0.0   
4  11.0   3.0   0.0   0.0   1.0   0.0   0.0   4.0   5.0   3.0   1.0   0.0   

   1125  1127  1129  1131  1133  2013  2016  2020  2050  2060  2068  2070  \
0   4.0   0.0   0.0   0.0   1.0   0.0   0.0   4.0   0.0   0.0   1.0   0.0   
1   3.0   0.0   0.0   0.0   0.0   0.0   0.0   6.0   0.0   0.0   0.0   0.0   
2   7.0   0.0   0.0   0.0   1.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   
3   7.0   0.0   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   1.0   
4  10.0   0.0   1.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   0.0   

   2090  2100  2105  2110  2122  2130  2150  2158  2164  2170  2180  2185  \
0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   
2   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   2188  2195  2198  2220  2230  2240  2261  2275  2282  2290  4001  4003  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0

In [28]:
df_master.to_excel('output/Destinations/dest_nycxco_daily_1009.xlsx')